In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import pathlib 
import os
import hyperopt as hp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import pathlib 
import os

from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from hyperopt import Trials, hp, fmin, tpe, STATUS_OK, space_eval 

import joblib
from sklearn.preprocessing import OneHotEncoder


C:\Users\jonat\AppData\Local\Temp\ipykernel_19256\2756606091.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
cur_path  = os.getcwd()

data_df = pd.read_csv(cur_path + '/account_data_info.csv')
history_df = pd.read_csv(cur_path + '/history.csv')
pd.set_option('display.min_rows', 150)


C:\Users\jonat\AppData\Local\Temp\ipykernel_19256\2169614156.py:4: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  history_df = pd.read_csv(cur_path + '/history.csv')


In [3]:
# deleting rows of account_data_info that dont appear in columns of history
filtered_data_df = data_df[data_df['Columns'].isin(history_df.columns)]

In [4]:
print(data_df.shape)
print(filtered_data_df.shape)
print(history_df.shape)

(111, 3)
(107, 3)
(673339, 106)


In [5]:
# show which rows were deleted
deleted_columns = data_df[~data_df['Columns'].isin(history_df.columns)]['Columns'].tolist()

print(deleted_columns)

print(history_df.isna().sum())

['credit_code', 'closing_date', 'closing_date', 'closure_reason']
id                                         0
type_code                                237
is_registered                              0
country_code                          669050
currency_code                              0
is_active                                  0
class_id                                3198
debit_code                              3198
last_trade_date                       307875
contract_type                          95823
inception_date                          3198
net_of_fees                                0
cashflows_custody_fee                 673339
fee_paid_separately                        0
custody_fee_withdrawal                     0
is_fee_exempt                              0
branch                                  3198
include_client_consolidation               0
use_client_address                         0
credit_limit_type                       3198
retail_plan                       

In [6]:
# determine columns with more than 95 percent NaN value
threshold_percent = 100

threshold_count = (threshold_percent / 100) * len(history_df)

missing_values_mask = history_df.isna()

missing_values_count = missing_values_mask.sum()

columns_with_more_than_x_missing = missing_values_count[missing_values_count > threshold_count].index.tolist()

print(columns_with_more_than_x_missing)
print(len(columns_with_more_than_x_missing))

[]
0


In [7]:
# checking whether column with >90% NaN important or not
"""
NaN_column = 'resp_specimen_plan'   #replace with >90% Column

df_with_nan_values = history_df[history_df[NaN_column].isna()]

percentage_churn = (df_with_nan_values['label'] == 'Churn').mean() * 100
percentage_no_churn = (df_with_nan_values['label'] == 'No Churn').mean() * 100

print("CHURN PERCENTAGE", percentage_churn)
print("NO CHURN PERCENTAGE", percentage_no_churn)

"""

## trying to create table for all

label_col = 'label'

for i in columns_with_more_than_x_missing:
    df_with_nan_values = history_df[history_df[i].isna()]
    percentage_churn = (df_with_nan_values[label_col] == 'Churn').mean() * 100
    percentage_no_churn = (df_with_nan_values[label_col] == 'No Churn').mean() * 100

    print('Column:', i)
    print("Percentage Churn:", percentage_churn)
    print("Percentage No Churn:", percentage_no_churn)
    print(" ")





In [8]:
# Total churn no churn percentage

percentage_churn = (history_df['label'] == 'Churn').mean() * 100
percentage_no_churn = (history_df['label'] == 'No Churn').mean() * 100
print("Total Chrun No Churn Percentage:")
print("Percentage Churn:", percentage_churn)
print("Percentage No Churn:", percentage_no_churn)



Total Chrun No Churn Percentage:
Percentage Churn: 79.96625770971234
Percentage No Churn: 20.033742290287655


In [9]:
# determining all date colunns and dropping

date_columns = ['last_trade_date', 'inception_date', 'last_update_date', 'non_calendar_year_end', 'plan_effective_date', 'plan_end_date', 'rrif_original_date', 'inserted_at', 'updated_at', 'retail_last_maintenance_time', 'last_maintenance_time']
history_df.drop(columns=date_columns, inplace=True)

# dropping select columns
history_df.drop(columns='country_code', inplace=True)
history_df.drop(columns='cashflows_custody_fee', inplace=True)

# just too many NaN dont care
too_many_NaN = ['arp_pension_origin', 'sss_location', 'sss_type', 'sss_agent', 'special_fee_code']
history_df.drop(columns=too_many_NaN, inplace = True)

# irrelevant
irrelevant = ['terminal_code', 'last_maintenance_user', 'retail_last_maintenance_user']
history_df.drop(columns=irrelevant, inplace=True)

# #test
test = ['risk_tolerance', 'investment_objective']
history_df.drop(columns=test, inplace=True)


In [10]:
# removing object value and filling NaN's

# bool columns 
for i in history_df.columns:
    if 'f' in history_df[i].values:
        history_df[i] = history_df[i].replace({'f': False, 't': True}).fillna(True)

# number columns
history_df[history_df.select_dtypes(include=['number']).columns] = history_df.select_dtypes(include=['number']).astype('float64')
def replace_nan_with_mode(column):
    if column.dtype == 'float64':
        mode_values = column.mode()
        if not mode_values.empty:
            mode_value = mode_values.iloc[0]
            return column.fillna(mode_value)
    return column

history_df = history_df.apply(replace_nan_with_mode)

# whats left should be str, fill with 'UNKNOWN'
history_df.fillna('UNKNOWN', inplace=True)


C:\Users\jonat\AppData\Local\Temp\ipykernel_19256\3861104044.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'f' in history_df[i].values:
C:\Users\jonat\AppData\Local\Temp\ipykernel_19256\3861104044.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  history_df[i] = history_df[i].replace({'f': False, 't': True}).fillna(True)
C:\Users\jonat\AppData\Local\Temp\ipykernel_19256\3861104044.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  history_df[i] = his

In [14]:
pd.set_option('display.max_columns', None)

need_encode = history_df.drop('label', axis=1).select_dtypes('object').columns
objects = history_df.select_dtypes('object')

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(objects)
codes = enc.transform(objects).toarray()
feature_names = enc.get_feature_names(need_encode)

history_df = enc.transform(history_df)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 673339 entries, 0 to 673338
Data columns (total 17 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   type_code                    673339 non-null  object
 1   currency_code                673339 non-null  object
 2   debit_code                   673339 non-null  object
 3   branch                       673339 non-null  object
 4   retail_plan                  673339 non-null  object
 5   language_code                673339 non-null  object
 6   is_midwest_clearing_account  673339 non-null  object
 7   iso_funds_code               673339 non-null  object
 8   shareholder_language         673339 non-null  object
 9   dup_trip_quad_code           673339 non-null  object
 10  special_tag                  673339 non-null  object
 11  conjunction                  673339 non-null  object
 12  title                        673339 non-null  object
 13  function_code 

TypeError: Encoders require their input argument must be uniformly strings or numbers. Got ['bool', 'str']

In [ ]:
print(history_df.head(5))
print(history_df.shape)

          id  is_registered  is_active  class_id  contract_type  net_of_fees  \
0   893724.0          False       True       3.0           17.0        False   
1  1268094.0           True       True       5.0           18.0        False   
2   606613.0           True       True       5.0           16.0        False   
3   741930.0          False       True       3.0           12.0        False   
4  1137922.0          False       True       3.0           17.0        False   

   fee_paid_separately  custody_fee_withdrawal  is_fee_exempt  \
0                False                   False          False   
1                False                   False          False   
2                False                   False          False   
3                False                   False          False   
4                False                   False          False   

   include_client_consolidation  use_client_address  credit_limit_type  \
0                          True               False   

In [ ]:
cur_path = cur_path + '/processed.csv'

history_df.to_csv(cur_path, index=False)


In [ ]:
cur_path  = os.getcwd()

df = pd.read_csv(cur_path + '/processed.csv')
pd.set_option('display.min_rows', 150)


In [ ]:
df['label'] = df['label'].eq('Churn').mul(1)

X = df.drop("label", axis = 1)
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# space = {
#     'n_estimators': hp.choice('n_estimators', range(50, 500)),
#     'max_depth': hp.choice('max_depth', [1, 5, 10, 20, 50, 75, 100, 150, 200]),
#     'min_samples_split': hp.choice('min_samples_split', [2, 3, 4, 5, 10, 20]),
#     'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 3, 4, 5]),
#     'bootstrap': hp.choice('bootstrap', [True, False]),
#     'criterion': hp.choice('criterion', ['gini', 'entropy']),
#     'max_features': hp.choice('max_features', ['sqrt', 'log2', None])
# }

# kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)

# def objective(params):
#     params['max_depth'] = int(params['max_depth'])
#     clf = RandomForestClassifier(**params)
#     print(params)
#     scores = cross_val_score(clf, X_train, y_train, cv = kfold, scoring = 'f1', n_jobs = -1)
#     print(scores)
#     best_score = mean(scores) 
#     loss = -best_score
#     return {'loss': loss, 'params': params, 'status': STATUS_OK}

# num_trials = Trials()
# best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 40, trials = num_trials)

randomForest = RandomForestClassifier(bootstrap= False, criterion= 'gini', max_depth = 75, max_features = 'sqrt', min_samples_leaf = 3, min_samples_split = 10, n_estimators = 159)
randomForest.fit(X_train, y_train)

y_pred = randomForest.predict(X_test)


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
f1_score = f1_score(y_test, y_pred)

print('Accuracy', accuracy)
print('F1 Score', f1_score)
print('Classification Report:\n', classification_report(y_test, y_pred))

Accuracy 0.951369293373333
F1 Score 0.9694512988678928
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.89      0.88     27180
           1       0.97      0.97      0.97    107488

    accuracy                           0.95    134668
   macro avg       0.92      0.93      0.93    134668
weighted avg       0.95      0.95      0.95    134668



In [ ]:
joblib.dump(randomForest, 'randomForest.pkl')

### Predictions

In [15]:
model = joblib.load('randomForest.pkl')

test_df = pd.read_csv('test.csv')

# determining all date colunns and dropping

date_columns = ['last_trade_date', 'inception_date', 'last_update_date', 'non_calendar_year_end', 'plan_effective_date', 'plan_end_date', 'rrif_original_date', 'inserted_at', 'updated_at', 'retail_last_maintenance_time', 'last_maintenance_time']
test_df.drop(columns=date_columns, inplace=True)

# dropping select columns
test_df.drop(columns='country_code', inplace=True)
test_df.drop(columns='cashflows_custody_fee', inplace=True)

# just too many NaN dont care
too_many_NaN = ['arp_pension_origin', 'sss_location', 'sss_type', 'sss_agent', 'special_fee_code']
test_df.drop(columns=too_many_NaN, inplace = True)

# irrelevant
irrelevant = ['terminal_code', 'last_maintenance_user', 'retail_last_maintenance_user']
test_df.drop(columns=irrelevant, inplace=True)

# #test
test = ['risk_tolerance', 'investment_objective']
test_df.drop(columns=test, inplace=True)


C:\Users\jonat\AppData\Local\Temp\ipykernel_32104\2260750717.py:3: DtypeWarning: Columns (62,64,87) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('test.csv')


In [16]:
# removing object value and filling NaN's

# bool columns 
for i in test_df.columns:
    if 'f' in test_df[i].values:
        test_df[i] = test_df[i].replace({'f': False, 't': True}).fillna(True)

# number columns
test_df[test_df.select_dtypes(include=['number']).columns] = test_df.select_dtypes(include=['number']).astype('float64')
def replace_nan_with_mode(column):
    if column.dtype == 'float64':
        mode_values = column.mode()
        if not mode_values.empty:
            mode_value = mode_values.iloc[0]
            return column.fillna(mode_value)
    return column

test_df = test_df.apply(replace_nan_with_mode)

# whats left should be str, fill with 'UNKNOWN'
test_df.fillna('UNKNOWN', inplace=True)


C:\Users\jonat\AppData\Local\Temp\ipykernel_32104\420565598.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'f' in test_df[i].values:
C:\Users\jonat\AppData\Local\Temp\ipykernel_32104\420565598.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df[i] = test_df[i].replace({'f': False, 't': True}).fillna(True)
C:\Users\jonat\AppData\Local\Temp\ipykernel_32104\420565598.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df[i] = test_df[i].replace

In [17]:
# need_encode = test_df.select_dtypes(exclude=['bool', 'float64']).columns

test_df = enc.transform(test_df)

In [18]:
print(test_df.info(verbose = True))
preprocessed = pd.read_csv('processed.csv')
print(preprocessed.info(verbose = True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168335 entries, 0 to 168334
Data columns (total 240 columns):
 #    Column                                 Dtype  
---   ------                                 -----  
 0    id                                     float64
 1    is_registered                          bool   
 2    is_active                              bool   
 3    class_id                               float64
 4    contract_type                          float64
 5    net_of_fees                            bool   
 6    fee_paid_separately                    bool   
 7    custody_fee_withdrawal                 bool   
 8    is_fee_exempt                          bool   
 9    include_client_consolidation           bool   
 10   use_client_address                     bool   
 11   credit_limit_type                      float64
 12   is_spousal                             bool   
 13   is_arp_locked                          bool   
 14   dividend_confirm_code             

In [19]:
predict_df = test_df[['id']]
pred = model.predict(test_df)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- function_code_ES QUALITE POUR
- function_code_IN TRUST OF
- is_midwest_clearing_account_True
- portfolio_name_address_option_0
- portfolio_name_address_option_0.0
- ...
Feature names seen at fit time, yet now missing:
- branch_IND ALLI
- conjunction_OR
- debit_code_1
- debit_code_7
- debit_code_B
- ...
